In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
%matplotlib inline

In [4]:
file1 = open("../../data/features/2019_05_17_TSLA_y_08_yishuang.pickle",'rb')
tsla = pickle.load(file1, encoding='utf-8')

file2 = open("../../data/features/2019_05_17_all_features.pickle",'rb')
elon = pickle.load(file2, encoding='utf-8')

In [4]:
elon.columns

Index(['id', 'Datetime', 'text', 'sentiment', 'subjectivity', 'isreply',
       'year', 'month', 'day', 'date', 'DayofWeek', 'Weekend', 'Time',
       'BinaryTrading', 'DayDistance', 'gtrend', 'CommentSentimental',
       'CommentSubjectivity', 'countComment', 'posCommentSum', 'negCommentSum',
       'CommentSD', 'posCommentSD', 'negCommentSD', 'negCommentPercent',
       'posCommentPercent'],
      dtype='object')

In [6]:
tsla.columns

Index(['Hour2', 'Hour3', 'Hour4', 'Hour5', 'Hour6', 'Hour7', 'Date',
       'Daily_volume', 'Datetime', 'Hour_volume', 'Trading_Hour',
       'Roll_volume', 'Moving_average_5', 'Volume_perc', 'Perc_shift_7',
       'Perc_shift_14', 'Perc_shift_21', 'Perc_shift_28', 'Perc_shift_35',
       'Perc_shift_42', 'Perc_shift_49', 'AR_pred', 'Residual'],
      dtype='object')

In [7]:
# Add start and end datetime
tsla["Start_Datetime"] = tsla["Datetime"]
tsla['End_Datetime'] = tsla['Datetime'] + pd.DateOffset(hours=1)

In [8]:
tsla.head()

,Hour2,Hour3,Hour4,Hour5,Hour6,Hour7,Date,Daily_volume,Datetime,Hour_volume,...,Perc_shift_14,Perc_shift_21,Perc_shift_28,Perc_shift_35,Perc_shift_42,Perc_shift_49,AR_pred,Residual,Start_Datetime,End_Datetime
0,0,0,0,0,0,0,2016-06-06,2249500,2016-06-06 13:30:00+00:00,802995.0,...,0.191230,0.497019,0.129291,0.288979,0.333049,0.250512,0.180036,0.209508,2016-06-06 13:30:00+00:00,2016-06-06 14:30:00+00:00
1,1,0,0,0,0,0,2016-06-06,2249500,2016-06-06 14:30:00+00:00,279458.0,...,0.121435,0.274092,0.093569,0.162169,0.276517,0.241683,0.174876,-0.043552,2016-06-06 14:30:00+00:00,2016-06-06 15:30:00+00:00
2,0,1,0,0,0,0,2016-06-06,2249500,2016-06-06 15:30:00+00:00,292325.0,...,0.082991,0.140525,0.074788,0.078377,0.151645,0.095730,0.113209,0.018052,2016-06-06 15:30:00+00:00,2016-06-06 16:30:00+00:00
3,0,0,1,0,0,0,2016-06-06,2249500,2016-06-06 16:30:00+00:00,170419.0,...,0.086119,0.081911,0.066586,0.059976,0.075488,0.104234,0.087077,-0.012592,2016-06-06 16:30:00+00:00,2016-06-06 17:30:00+00:00
4,0,0,0,1,0,0,2016-06-06,2249500,2016-06-06 17:30:00+00:00,214303.0,...,0.091694,0.069936,0.078209,0.090399,0.085611,0.157451,0.097929,-0.006690,2016-06-06 17:30:00+00:00,2016-06-06 18:30:00+00:00


In [9]:
tsla["Start_Datetime"] = pd.to_datetime(tsla["Start_Datetime"], utc = True)
tsla["End_Datetime"] = pd.to_datetime(tsla["End_Datetime"], utc = True)
elon["Datetime"] = pd.to_datetime(elon["Datetime"], utc = True)

In [10]:
elon_new = elon[(elon['Datetime'] >= tsla.iloc[0]['Start_Datetime']) & (elon['Datetime'] <= tsla.iloc[-1]['End_Datetime'])]

In [11]:
elon_new.tail()

,id,Datetime,text,sentiment,subjectivity,isreply,keyword,year,month,day,...,CommentSentimental,CommentSubjectivity,countComment,posCommentSum,negCommentSum,CommentSD,posCommentSD,negCommentSD,negCommentPercent,posCommentPercent
4685,740723195431538689,2016-06-09 01:52:00+00:00,something about a flying metal suit...http://w...,0.000,0.000000,0,False,2016,6,9,...,16.864362,44.666755,181,21.462973,-4.598611,0.270031,0.290042,0.197206,0.121547,0.303867
4686,740540471081062401,2016-06-08 13:46:00+00:00,yes,0.000,0.000000,1,False,2016,6,8,...,19.660620,43.649106,179,21.525684,-1.865064,0.229870,0.236714,0.236278,0.055866,0.363128
4687,740387141591535616,2016-06-08 03:37:00+00:00,s and x cells are also panasonic,0.000,0.000000,1,False,2016,6,8,...,11.912356,27.576966,123,13.558189,-1.645833,0.238247,0.263092,0.270615,0.056911,0.333333
4688,740383850820296707,2016-06-08 03:24:00+00:00,would like to clarify that tesla is working ex...,0.000,0.000000,0,True,2016,6,8,...,11.134578,26.082522,117,12.780411,-1.645833,0.241406,0.269885,0.270615,0.059829,0.324786
4689,740296489532948480,2016-06-07 21:36:00+00:00,fourth rocket arrives in the hangar. aiming fo...,0.125,0.166667,0,False,2016,6,7,...,13.823846,26.797078,94,14.807179,-0.983333,0.233470,0.202021,0.097420,0.063830,0.446809


In [12]:
tweet_datetime = elon_new.iloc[0]['Datetime']
subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
subset_df

,Hour2,Hour3,Hour4,Hour5,Hour6,Hour7,Date,Daily_volume,Datetime,Hour_volume,...,Perc_shift_14,Perc_shift_21,Perc_shift_28,Perc_shift_35,Perc_shift_42,Perc_shift_49,AR_pred,Residual,Start_Datetime,End_Datetime
4976,0,0,0,0,0,1,2019-04-02,5478900,2019-04-02 19:30:00+00:00,855794.0,...,0.165635,0.117097,0.110133,0.078929,0.087253,0.132812,0.123796,0.020216,2019-04-02 19:30:00+00:00,2019-04-02 20:30:00+00:00


In [13]:
# y_target: volume percentage
volume_percs = []
trading_hour = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume percentage of first row of subset df
    perc = subset_df.iloc[0]['Volume_perc']
    hour = subset_df.iloc[0]['Trading_Hour']
    
    # append to list
    volume_percs.append(perc)
    trading_hour.append(int(hour))

elon_new['Volume_perc'] = pd.Series(volume_percs)
elon_new['Trading_Hour'] = pd.Series(trading_hour)

In [14]:
elon_new.head()

,id,Datetime,text,sentiment,subjectivity,isreply,keyword,year,month,day,...,countComment,posCommentSum,negCommentSum,CommentSD,posCommentSD,negCommentSD,negCommentPercent,posCommentPercent,Volume_perc,Trading_Hour
162,1113173498384441344,2019-04-02 20:16:00+00:00,dogecoin rulzpic.twitter.com/flwwugaglu,0.00,0.0000,0,False,2019,4,2,...,757,65.808511,-9.349753,0.215552,0.252742,0.181753,0.059445,0.229855,0.307817,1.0
163,1113164389929160706,2019-04-02 19:40:00+00:00,uh oh,0.00,0.0000,1,False,2019,4,2,...,125,11.632089,-1.938889,0.219263,0.242682,0.245950,0.064000,0.288000,0.307817,1.0
164,1113009339743100929,2019-04-02 09:24:00+00:00,dogecoin might be my fav cryptocurrency. it’s ...,0.15,0.4125,1,False,2019,4,2,...,29,3.081250,-0.454167,0.198969,0.178780,0.142542,0.103448,0.344828,0.307817,1.0
165,1113008497006804992,2019-04-02 09:21:00+00:00,yuphttps://twitter.com/nasa/status/11128601960...,0.00,0.0000,0,False,2019,4,2,...,243,20.762614,-5.672222,0.222568,0.223982,0.190050,0.065844,0.234568,0.307817,1.0
166,1113000800526036992,2019-04-02 08:50:00+00:00,some challenges with ice formation in the cryo...,0.00,0.0000,1,False,2019,4,2,...,76,3.768669,-2.215972,0.179058,0.229078,0.171248,0.144737,0.184211,0.307817,1.0


In [15]:
elon_new.groupby('Trading_Hour')['Trading_Hour'].count()

Trading_Hour
1.0    3316
2.0     122
3.0     137
4.0     204
5.0     214
6.0     212
7.0     161
Name: Trading_Hour, dtype: int64

In [16]:
tsla.head()

,Hour2,Hour3,Hour4,Hour5,Hour6,Hour7,Date,Daily_volume,Datetime,Hour_volume,...,Perc_shift_14,Perc_shift_21,Perc_shift_28,Perc_shift_35,Perc_shift_42,Perc_shift_49,AR_pred,Residual,Start_Datetime,End_Datetime
0,0,0,0,0,0,0,2016-06-06,2249500,2016-06-06 13:30:00+00:00,802995.0,...,0.191230,0.497019,0.129291,0.288979,0.333049,0.250512,0.180036,0.209508,2016-06-06 13:30:00+00:00,2016-06-06 14:30:00+00:00
1,1,0,0,0,0,0,2016-06-06,2249500,2016-06-06 14:30:00+00:00,279458.0,...,0.121435,0.274092,0.093569,0.162169,0.276517,0.241683,0.174876,-0.043552,2016-06-06 14:30:00+00:00,2016-06-06 15:30:00+00:00
2,0,1,0,0,0,0,2016-06-06,2249500,2016-06-06 15:30:00+00:00,292325.0,...,0.082991,0.140525,0.074788,0.078377,0.151645,0.095730,0.113209,0.018052,2016-06-06 15:30:00+00:00,2016-06-06 16:30:00+00:00
3,0,0,1,0,0,0,2016-06-06,2249500,2016-06-06 16:30:00+00:00,170419.0,...,0.086119,0.081911,0.066586,0.059976,0.075488,0.104234,0.087077,-0.012592,2016-06-06 16:30:00+00:00,2016-06-06 17:30:00+00:00
4,0,0,0,1,0,0,2016-06-06,2249500,2016-06-06 17:30:00+00:00,214303.0,...,0.091694,0.069936,0.078209,0.090399,0.085611,0.157451,0.097929,-0.006690,2016-06-06 17:30:00+00:00,2016-06-06 18:30:00+00:00


In [17]:
# adding AR-y
ar_y = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row of subset df
    y = subset_df.iloc[0]['AR_pred']
    

    # append to list
    ar_y.append(y)

elon_new['AR_pred'] = pd.Series(ar_y)

In [18]:
elon_new

,id,Datetime,text,sentiment,subjectivity,isreply,keyword,year,month,day,...,posCommentSum,negCommentSum,CommentSD,posCommentSD,negCommentSD,negCommentPercent,posCommentPercent,Volume_perc,Trading_Hour,AR_pred
162,1113173498384441344,2019-04-02 20:16:00+00:00,dogecoin rulzpic.twitter.com/flwwugaglu,0.000000,0.000000,0,False,2019,4,2,...,65.808511,-9.349753,0.215552,0.252742,1.817530e-01,0.059445,0.229855,0.307817,1.0,0.225384
163,1113164389929160706,2019-04-02 19:40:00+00:00,uh oh,0.000000,0.000000,1,False,2019,4,2,...,11.632089,-1.938889,0.219263,0.242682,2.459504e-01,0.064000,0.288000,0.307817,1.0,0.225384
164,1113009339743100929,2019-04-02 09:24:00+00:00,dogecoin might be my fav cryptocurrency. it’s ...,0.150000,0.412500,1,False,2019,4,2,...,3.081250,-0.454167,0.198969,0.178780,1.425422e-01,0.103448,0.344828,0.307817,1.0,0.225384
165,1113008497006804992,2019-04-02 09:21:00+00:00,yuphttps://twitter.com/nasa/status/11128601960...,0.000000,0.000000,0,False,2019,4,2,...,20.762614,-5.672222,0.222568,0.223982,1.900499e-01,0.065844,0.234568,0.307817,1.0,0.225384
166,1113000800526036992,2019-04-02 08:50:00+00:00,some challenges with ice formation in the cryo...,0.000000,0.000000,1,False,2019,4,2,...,3.768669,-2.215972,0.179058,0.229078,1.712479e-01,0.144737,0.184211,0.307817,1.0,0.225384
167,1112999601332641792,2019-04-02 08:45:00+00:00,what could possibly go wrong?,-0.500000,0.900000,1,False,2019,4,2,...,3.606764,-2.215972,0.179502,0.236255,1.712479e-01,0.146667,0.173333,0.307817,1.0,0.225384
168,1112882387963195393,2019-04-02 01:00:00+00:00,no,0.000000,0.000000,1,False,2019,4,2,...,12.547262,-5.050643,0.233543,0.221653,2.030256e-01,0.133333,0.213333,0.307817,1.0,0.225384
169,1112848471877283841,2019-04-01 22:45:00+00:00,you’re so right. working on it!,0.142857,0.267857,1,False,2019,4,1,...,2.571549,-1.487500,0.210355,0.148577,2.640431e-01,0.138889,0.277778,0.307817,1.0,0.225384
170,1112847351595466753,2019-04-01 22:40:00+00:00,that car seemed so advanced when i watched tha...,0.250000,0.300000,1,True,2019,4,1,...,5.412311,0.000000,0.292040,0.265866,0.000000e+00,0.000000,0.480000,0.307817,1.0,0.225384
171,1112840482856480768,2019-04-01 22:13:00+00:00,and even tinier ones for tardigrades,0.000000,0.000000,1,False,2019,4,1,...,20.759905,-4.820757,0.225421,0.240173,1.228506e-01,0.136585,0.312195,0.307817,1.0,0.225384


In [19]:
elon_new.dropna(inplace = True)

In [21]:
with open('../../data/2019_05_17_ar_pred.pickle', 'wb') as handle:
        pickle.dump(elon_new, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
# y_target 2: volume residual
residuals = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row of subset df
    residual = subset_df.iloc[0]['Residual']
    

    # append to list
    residuals.append(residual)

elon_new['Residual'] = pd.Series(residuals)

In [ ]:
with open('../../data/features/2019_05_17_features_finally.pickle', 'wb') as handle:
        pickle.dump(elon_new, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
# rolling average volume percentage
roll_vol_perc = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume of first row of subset df
    vol_perc = subset_df.iloc[0]['Rolling_perc_volume']
 
    # append to list
    roll_vol_perc.append(vol_perc)

elon_new['Rolling_volume_perc'] = pd.Series(roll_vol_perc)

KeyError: 'Rolling_perc_volume'

In [21]:
# rolling average volume 
roll_vol = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume of first row of subset df
    vol = subset_df.iloc[0]['Rolling_volume']
 
    # append to list
    roll_vol.append(vol)

elon_new['Rolling_volume'] = pd.Series(roll_vol)

KeyError: 'Rolling_volume'

In [ ]:
# y_target 1: volume
volumes = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume of first row of subset df
    volume = subset_df.iloc[0]['Volume']
 
    # append to list
    volumes.append(volume)

elon_new['That_hour_volume'] = pd.Series(volumes)

In [ ]:
# y_target 2: volume percentage
volume_percs = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume percentage of first row of subset df
    perc = subset_df.iloc[0]['Percentage_Volume']

    # append to list
    volume_percs.append(perc)

elon_new['That_hour_volume_perc'] = pd.Series(volume_percs)

In [ ]:
# y_target 3: volume percentage residual
residual_perc = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row of subset df
    residual = subset_df.iloc[0]['Res_Perc_vol']
    

    # append to list
    residual_perc.append(residual)

elon_new['That_hour_res_vol_perc'] = pd.Series(residual_perc)

In [ ]:
# y_target 4: volume residual
residuals = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row of subset df
    residual = subset_df.iloc[0]['Res_vol']
    

    # append to list
    residuals.append(residual)

elon_new['That_hour_res_vol'] = pd.Series(residuals)

In [ ]:
elon_new.drop(['Datetime_after_1h'], axis = 1, inplace = True)

In [ ]:
elon_new.head(10)

In [ ]:
# y_target 5: volume in 2 hours
volumes = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume of first row and second row of subset df, then take average
    volume = (subset_df.iloc[0]['Volume'] + subset_df.iloc[1]['Volume'])/2
 
    # append to list
    volumes.append(volume)

elon_new['Volume_in_2h'] = pd.Series(volumes)

In [ ]:
# y_target 6: volume percentage in 2 hours
volume_percs = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']

    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly volume percentage of first row and second row of subset df, then take average
    perc = (subset_df.iloc[0]['Percentage_Volume'] + subset_df.iloc[1]['Percentage_Volume'])/2

    # append to list
    volume_percs.append(perc)

elon_new['Volume_perc_in_2h'] = pd.Series(volume_percs)

In [ ]:
# y_target 7: volume percentage residual in 2 hours
residual_perc = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row and second row of subset df and take average
    residual = (subset_df.iloc[0]['Res_Perc_vol'] + subset_df.iloc[0]['Res_Perc_vol'])/2

    # append to list
    residual_perc.append(residual)

elon_new['Res_vol_perc_in_2h'] = pd.Series(residual_perc)

In [ ]:
# y_target 8: volume residual in 2 hours
residuals = []

for i, row in elon_new.iterrows():

    tweet_datetime = row['Datetime']
    
    subset_df = tsla[tsla['End_Datetime']>tweet_datetime]
    # get hourly residual of first row and second row of subset df and take average
    residual = (subset_df.iloc[0]['Res_vol'] + subset_df.iloc[0]['Res_vol'])/2
    

    # append to list
    residuals.append(residual)

elon_new['Res_vol_in_2h'] = pd.Series(residuals)

In [ ]:
elon_new.head()

In [ ]:
with open('../../data/features/2019_04_30_features_with_8y.pickle', 'wb') as handle:
        pickle.dump(elon_new, handle, protocol=pickle.HIGHEST_PROTOCOL)